# Sage Shaw

### ME 571 - Spring 2018

# HW 2 - Problem 1

# Using a Pool to compute a matrix-matrix multiply

<hr/>

Your job in this problem is to improve the speed of the matrix-matrix multiply, implemented here using a Pool.  
You are free to be creative in writing your matrix-matrix multiply, but must follow a few ground rules. 

* You must use one of the Pool functions (map, map_async, apply or apply_async). 

* You cannot use any global variables

* You are free to re-write the way in which the matrix is distributed to processors.  For example, you can try tiling individual dot products to get better cache performance. 

* Your answer should agree with the `numpy.matmul` result to within about $10^{-10}$.  The code below computes the difference for you. 

To show the speed-up you get, write a second function (using `def`) that computes your faster matrix-matrix multiply and then report both the original results and your faster results.  

We are not looking for the "fastest" method to compute a matrix-matrix product (Numpy's `matmul` beats everything).  We are also not looking for parallel speed-up.   Rather, this problem should give you a better feel as to how the multiprocessing Pool module works, and how to use it optimally.

If you are successful, you should see about a factor 5-10 (or more) speedup.  Try out your matrix-matrix multiply on matrices of dimensions $N = 2^7, 2^8, 2^9$ and $2^{10}$. 

To see how the Pool module works, see notebook posted on the course Wiki (Tuesday, Week 04). 

In [3]:
from multiprocessing import Pool
import numpy as np
import numpy.linalg as la
import itertools
import pandas as pd

In [4]:
#Original Version 0
def dot_ij(d):
    i,j,x,y = d
    return (i,j,np.dot(x,y))

def matmul_pool(A,B):
    m1,n1 = A.shape
    m2,n2 = B.shape
    if n1 != m2:
        print("Inner matrix dimensions do not agree")
        return None
    
    Bt = np.transpose(B)
    
    map_list = [(i,j,A[i],Bt[j]) for (i,j) in itertools.product(range(0,m1),range(0,n2))]    
    
    pool = Pool()
    results = []
    for d in map_list:
        r = pool.apply(func=dot_ij,args=(d,))
        results.append(r)

    C = np.empty((m1,n2),dtype='d')
    for r in results:
        i,j,d = r
        C[i][j] = d
        
    pool.close() #This line was not in the original       
    return C

In [5]:
def gen_mat(N):
    np.random.seed(1234)
    A = np.random.rand(N,N)
    B = np.random.rand(N,N)
    return A, B

In [9]:
N = 2**6
A, B = gen_mat(N)
n_loop = 1
r_rep = 1
tr = %timeit -n $n_loop -r $r_rep -o pass; C_pool = matmul_pool(A,B)
time_loop = tr.best

# Get answer that we can compare to the numpy result
C_pool = matmul_pool(A,B)

print("")
print("{:>25s} {:12d}".format("Matrix dimensions (N)",N))
print("{:>25s} {:12.4e}".format("norm(C_pool-C_true)",la.norm(C_pool-np.matmul(A,B))))
print("{:>25s} {:12.6f}".format("Time using Pool (s)", time_loop))

1 loop, best of 1: 1.84 s per loop

    Matrix dimensions (N)           64
      norm(C_pool-C_true)   2.3573e-13
      Time using Pool (s)     1.837768


In [41]:
#New Version 1
def my_dot(args):
    A,b = args
    return np.dot(A,b)

def matmul_pool1(A,B):
    N = len(A)
    assert len(A)==len(B[0])
    Bcols = [B[:,i:i+1] for i in range(N)]
    pool = Pool()
    cols = pool.map(my_dot,[(A,b) for b in Bcols])
    C = np.array(cols).T #.reshape(N,N)
    pool.close()            
    return C

In [42]:
N = 2**6
A,B = gen_mat(N)
n_loop = 1
r_rep = 1
tr = %timeit -n $n_loop -r $r_rep -o pass; C_pool = matmul_pool1(A,B)
time_loop1 = tr.best

# Get answer that we can compare to the numpy result
C_pool = matmul_pool1(A,B)

print("")
print("{:>25s} {:12d}".format("Matrix dimensions (N)",N))
print("{:>25s} {:12.4e}".format("norm(C_pool-C_true)",la.norm(C_pool-np.matmul(A,B))))
print("{:>25s} {:12.6f}".format("Time using Pool (s)", time_loop1))

print('The speed up is: %f' % (time_loop/time_loop1))

1 loop, best of 1: 225 ms per loop

    Matrix dimensions (N)           64
      norm(C_pool-C_true)   2.6523e-13
      Time using Pool (s)     0.224873
The speed up is: 23.326588


In [43]:
Ns = [2**i for i in range(7,11)]
for N in Ns:
    A, B = gen_mat(N)
    n_loop = 1
    r_rep = 1
    tr = %timeit -n $n_loop -r $r_rep -o pass; C_pool = matmul_pool1(A,B)
    time_loop = tr.best

    # Get answer that we can compare to the numpy result
    C_pool = matmul_pool1(A,B)

    print("")
    print("{:>25s} {:12d}".format("Matrix dimensions (N)",N))
    print("{:>25s} {:12.4e}".format("norm(C_pool-C_true)",la.norm(C_pool-np.matmul(A,B))))
    print("{:>25s} {:12.6f}".format("Time using Pool (s)", time_loop))

1 loop, best of 1: 520 ms per loop

    Matrix dimensions (N)          128
      norm(C_pool-C_true)   1.4765e-12
      Time using Pool (s)     0.520475
1 loop, best of 1: 1.08 s per loop

    Matrix dimensions (N)          256
      norm(C_pool-C_true)   8.0818e-12
      Time using Pool (s)     1.077260
1 loop, best of 1: 1.4 s per loop

    Matrix dimensions (N)          512
      norm(C_pool-C_true)   2.3101e-11
      Time using Pool (s)     1.400712
1 loop, best of 1: 5.11 s per loop

    Matrix dimensions (N)         1024
      norm(C_pool-C_true)   1.2156e-10
      Time using Pool (s)     5.111356


## Conclusion
The important differences were that the `pool.map` function was used instead of the `pool.apply` to allow the calculations to be done in parallel, and that the results were treated as vectors to avoid loops.